In [1]:
import os
import tensorflow as tf
import numpy as np

from basenetwork import set_flattened_weights
from derivativelayer import DerivativeLayer
from basenetwork import Network
from PINN import PINNModel
from optimizer import Optimizer
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.gridspec import GridSpec
from data_prep import data_prep
from plottingtools import plot_contour, ErrorAnalysisPlot
import pickle
"""
We will solve the equation between 0 and Xmax and 0 and Tmax
"""

#Global Parameters for training
N_col = 10000 #Number of collocation points
N_bound_ini = 100 #total number of boundary and initial condition points
N_test_points = 1000

basemodel = Network.basemodel()
GridStabilityPINN = PINNModel(basemodel).build()
x_train, y_train = data_prep(N_ini = 150)

optimizer = Optimizer(model=GridStabilityPINN, x_train=x_train, y_train=y_train,maxiter= 20)
# optimizer.fit()

In [2]:
optimizer.model(x_train)

[<tf.Tensor: shape=(100000000, 1), dtype=float32, numpy=
 array([[-0.00447456],
        [ 0.04625747],
        [ 0.04177046],
        ...,
        [ 0.06861031],
        [ 0.06492653],
        [ 0.06957439]], dtype=float32)>,
 <tf.Tensor: shape=(150, 1), dtype=float32, numpy=
 array([[0.11332276],
        [0.10048193],
        [0.10576363],
        [0.12097949],
        [0.14369392],
        [0.12840006],
        [0.07845022],
        [0.06279618],
        [0.1373223 ],
        [0.15543693],
        [0.0893613 ],
        [0.10473336],
        [0.14856464],
        [0.09602886],
        [0.08887982],
        [0.14417224],
        [0.14405233],
        [0.10344017],
        [0.0990976 ],
        [0.10096003],
        [0.07972481],
        [0.11019791],
        [0.14579406],
        [0.15296385],
        [0.09963814],
        [0.07407856],
        [0.11475661],
        [0.08969357],
        [0.13042761],
        [0.12813202],
        [0.1131299 ],
        [0.14343993],
        [0.08780947

In [3]:
inputs, _, _ = x_train

t = tf.convert_to_tensor(inputs[:,0], dtype = 'float32')  # Time
P = tf.convert_to_tensor(inputs[:,1], dtype = 'float32')  # Power
m = tf.convert_to_tensor(inputs[:,2], dtype = 'float32')  # Inertial constant of system
d = tf.convert_to_tensor(inputs[:,3], dtype = 'float32')  # Damping constant of generator

with tf.GradientTape(persistent=True) as gg:
    gg.watch(t)
    c = basemodel(tf.stack([t,P,m,d],axis=1))  # calculate output (i.e. concentration)
    dc_dt = gg.gradient(c, t)  # Derive within tape so you can use tape after 

d2c_dt2 = gg.gradient(dc_dt,t)



In [11]:
tf.math.multiply(t,dc_dt)

<tf.Tensor: shape=(10000,), dtype=float32, numpy=
array([ 0.03183849,  0.03726398,  0.05237292, ..., -0.037343  ,
        0.06269732,  0.04501588], dtype=float32)>